In [45]:
#NN on new york data set without normalizing the output, with merging latitude and longitude by h3-Uber function, to predict the **duration** 
#Project Title: Improving the Bus Transit System by Predicting the Bus Arrival time, Using Regression Models and Neural Networks
# Group Members: Sanaz Hossseini, Shruthi Pasumarthi, Narges Rashvand

In [2]:
import os
import pandas as pd
import time
from datetime import datetime
import torch
import numpy as np
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import torch.nn as nn
import torch.nn.functional as F
torch.set_printoptions(edgeitems=2, linewidth=75)
import math
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
#from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import torch.nn as nn
import pandas as pd

In [3]:
file = "MTA-1706-Edited-12112022.csv"
target_file = os.getcwd() + f"/{file}"

In [4]:
df = pd.read_csv(target_file)
df

,RecordedAtTime,OriginLat,OriginLong,DestinationLat,DestinationLong,VehicleLocation.Latitude,VehicleLocation.Longitude,DistanceFromStop,ExpectedArrivalTime,ScheduledArrivalTime
0,0:03:00,40.616104,-74.031143,40.656048,-73.907379,40.635170,-73.960803,76.0,0:03:00,0:06:14
1,0:03:00,40.643169,-74.073494,40.575935,-74.167686,40.590802,-74.158340,62.0,0:03:00,23:58:02
2,0:03:00,40.875008,-73.880142,40.912376,-73.902534,40.886010,-73.912647,5.0,0:03:00,0:00:53
3,0:03:00,40.701748,-73.802399,40.666012,-73.735939,40.668002,-73.729348,267.0,0:04:00,0:03:00
4,0:03:00,40.881187,-73.909340,40.809654,-73.928360,40.868134,-73.893032,11.0,0:03:00,23:59:38
...,...,...,...,...,...,...,...,...,...,...
9995,5:03:00,40.622311,-74.028687,40.640297,-74.131332,40.596955,-74.069769,90.0,5:03:00,5:02:12
9996,5:03:00,40.761806,-73.829559,40.692406,-73.745216,40.692326,-73.744289,318.0,5:04:00,4:22:13
9997,5:03:00,40.761806,-73.829559,40.692406,-73.745216,40.692326,-73.744289,318.0,5:04:00,4:20:25
9998,5:03:00,40.862228,-73.890442,40.877934,-73.866585,40.871502,-73.876412,185.0,5:03:00,5:01:25


In [5]:
df.dropna(axis = 0, how = 'any', thresh = None, subset = None, inplace = True) #The code, if we want to remove all the rows with empty cells.  
df

,RecordedAtTime,OriginLat,OriginLong,DestinationLat,DestinationLong,VehicleLocation.Latitude,VehicleLocation.Longitude,DistanceFromStop,ExpectedArrivalTime,ScheduledArrivalTime
0,0:03:00,40.616104,-74.031143,40.656048,-73.907379,40.635170,-73.960803,76.0,0:03:00,0:06:14
1,0:03:00,40.643169,-74.073494,40.575935,-74.167686,40.590802,-74.158340,62.0,0:03:00,23:58:02
2,0:03:00,40.875008,-73.880142,40.912376,-73.902534,40.886010,-73.912647,5.0,0:03:00,0:00:53
3,0:03:00,40.701748,-73.802399,40.666012,-73.735939,40.668002,-73.729348,267.0,0:04:00,0:03:00
4,0:03:00,40.881187,-73.909340,40.809654,-73.928360,40.868134,-73.893032,11.0,0:03:00,23:59:38
...,...,...,...,...,...,...,...,...,...,...
9995,5:03:00,40.622311,-74.028687,40.640297,-74.131332,40.596955,-74.069769,90.0,5:03:00,5:02:12
9996,5:03:00,40.761806,-73.829559,40.692406,-73.745216,40.692326,-73.744289,318.0,5:04:00,4:22:13
9997,5:03:00,40.761806,-73.829559,40.692406,-73.745216,40.692326,-73.744289,318.0,5:04:00,4:20:25
9998,5:03:00,40.862228,-73.890442,40.877934,-73.866585,40.871502,-73.876412,185.0,5:03:00,5:01:25


In [6]:
print(df.dtypes)

RecordedAtTime                object
OriginLat                    float64
OriginLong                   float64
DestinationLat               float64
DestinationLong              float64
VehicleLocation.Latitude     float64
VehicleLocation.Longitude    float64
DistanceFromStop             float64
ExpectedArrivalTime           object
ScheduledArrivalTime          object
dtype: object


In [7]:
print(df['RecordedAtTime'])
print(df['ExpectedArrivalTime'])
print(df['ScheduledArrivalTime'])

0       0:03:00
1       0:03:00
2       0:03:00
3       0:03:00
4       0:03:00
         ...   
9995    5:03:00
9996    5:03:00
9997    5:03:00
9998    5:03:00
9999    5:03:00
Name: RecordedAtTime, Length: 8933, dtype: object
0       0:03:00
1       0:03:00
2       0:03:00
3       0:04:00
4       0:03:00
         ...   
9995    5:03:00
9996    5:04:00
9997    5:04:00
9998    5:03:00
9999    5:03:00
Name: ExpectedArrivalTime, Length: 8933, dtype: object
0        0:06:14
1       23:58:02
2        0:00:53
3        0:03:00
4       23:59:38
          ...   
9995     5:02:12
9996     4:22:13
9997     4:20:25
9998     5:01:25
9999     5:04:00
Name: ScheduledArrivalTime, Length: 8933, dtype: object


In [8]:
df['RecordedAtTime'] = pd.to_datetime(df['RecordedAtTime'], errors = 'coerce')
tindex = pd.DatetimeIndex(df.RecordedAtTime) 
df['RecordedAtTime'] = tindex.strftime('%H:%M:%S')

In [9]:
df['RecordedAtTime'] = pd.to_timedelta(df['RecordedAtTime']).dt.total_seconds()
df['RecordedAtTime'] 

0         180.0
1         180.0
2         180.0
3         180.0
4         180.0
         ...   
9995    18180.0
9996    18180.0
9997    18180.0
9998    18180.0
9999    18180.0
Name: RecordedAtTime, Length: 8933, dtype: float64

In [10]:
df['ExpectedArrivalTime'] = pd.to_datetime(df['ExpectedArrivalTime'], errors = 'coerce')
tindex2 = pd.DatetimeIndex(df.ExpectedArrivalTime) 
df['ExpectedArrivalTime'] = tindex2.strftime('%H:%M:%S')

In [11]:
df['ExpectedArrivalTime'] = pd.to_timedelta(df['ExpectedArrivalTime']).dt.total_seconds()
df['ExpectedArrivalTime'] 

0         180.0
1         180.0
2         180.0
3         240.0
4         180.0
         ...   
9995    18180.0
9996    18240.0
9997    18240.0
9998    18180.0
9999    18180.0
Name: ExpectedArrivalTime, Length: 8933, dtype: float64

In [12]:
df['ScheduledArrivalTime'] = pd.to_datetime(df['ScheduledArrivalTime'], errors = 'coerce')
tindex3 = pd.DatetimeIndex(df.ScheduledArrivalTime) 
df['ScheduledArrivalTime'] = tindex3.strftime('%H:%M:%S')

In [13]:
df['ScheduledArrivalTime'] = pd.to_timedelta(df['ScheduledArrivalTime']).dt.total_seconds()
df['ScheduledArrivalTime'] 

0         374.0
1       86282.0
2          53.0
3         180.0
4       86378.0
         ...   
9995    18132.0
9996    15733.0
9997    15625.0
9998    18085.0
9999    18240.0
Name: ScheduledArrivalTime, Length: 8933, dtype: float64

In [14]:
df

,RecordedAtTime,OriginLat,OriginLong,DestinationLat,DestinationLong,VehicleLocation.Latitude,VehicleLocation.Longitude,DistanceFromStop,ExpectedArrivalTime,ScheduledArrivalTime
0,180.0,40.616104,-74.031143,40.656048,-73.907379,40.635170,-73.960803,76.0,180.0,374.0
1,180.0,40.643169,-74.073494,40.575935,-74.167686,40.590802,-74.158340,62.0,180.0,86282.0
2,180.0,40.875008,-73.880142,40.912376,-73.902534,40.886010,-73.912647,5.0,180.0,53.0
3,180.0,40.701748,-73.802399,40.666012,-73.735939,40.668002,-73.729348,267.0,240.0,180.0
4,180.0,40.881187,-73.909340,40.809654,-73.928360,40.868134,-73.893032,11.0,180.0,86378.0
...,...,...,...,...,...,...,...,...,...,...
9995,18180.0,40.622311,-74.028687,40.640297,-74.131332,40.596955,-74.069769,90.0,18180.0,18132.0
9996,18180.0,40.761806,-73.829559,40.692406,-73.745216,40.692326,-73.744289,318.0,18240.0,15733.0
9997,18180.0,40.761806,-73.829559,40.692406,-73.745216,40.692326,-73.744289,318.0,18240.0,15625.0
9998,18180.0,40.862228,-73.890442,40.877934,-73.866585,40.871502,-73.876412,185.0,18180.0,18085.0


In [15]:
print(df.dtypes)

RecordedAtTime               float64
OriginLat                    float64
OriginLong                   float64
DestinationLat               float64
DestinationLong              float64
VehicleLocation.Latitude     float64
VehicleLocation.Longitude    float64
DistanceFromStop             float64
ExpectedArrivalTime          float64
ScheduledArrivalTime         float64
dtype: object


In [16]:
l = df.shape[0]

In [17]:
arr = df.to_numpy()

In [18]:
import h3
origin_vehicle = np.zeros(l)

vehicle_destination = np.zeros(l)

for i in range(l):

    coords_1 = (arr[i, 1], arr[i, 2])

    coords_2 = (arr[i,5], arr[i,6])  

    origin_vehicle[i] = h3.point_dist(coords_1, coords_2, unit = 'm')
    
    coords_3 = (arr[i, 3], arr[i, 4])

    coords_4 = (arr[i,5], arr[i,6])

    vehicle_destination[i] = h3.point_dist(coords_1, coords_2, unit = 'm')

In [19]:
o = pd.DataFrame(origin_vehicle, columns = ['O_V'])

vd = pd.DataFrame(vehicle_destination, columns = ['V_D'])

In [20]:
df['O_V'] = o

df['V_D'] = vd

In [21]:
df

,RecordedAtTime,OriginLat,OriginLong,DestinationLat,DestinationLong,VehicleLocation.Latitude,VehicleLocation.Longitude,DistanceFromStop,ExpectedArrivalTime,ScheduledArrivalTime,O_V,V_D
0,180.0,40.616104,-74.031143,40.656048,-73.907379,40.635170,-73.960803,76.0,180.0,374.0,6303.540673,6303.540673
1,180.0,40.643169,-74.073494,40.575935,-74.167686,40.590802,-74.158340,62.0,180.0,86282.0,9230.043179,9230.043179
2,180.0,40.875008,-73.880142,40.912376,-73.902534,40.886010,-73.912647,5.0,180.0,53.0,2994.093405,2994.093405
3,180.0,40.701748,-73.802399,40.666012,-73.735939,40.668002,-73.729348,267.0,240.0,180.0,7212.611156,7212.611156
4,180.0,40.881187,-73.909340,40.809654,-73.928360,40.868134,-73.893032,11.0,180.0,86378.0,1996.682927,1996.682927
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,18180.0,40.622311,-74.028687,40.640297,-74.131332,40.596955,-74.069769,90.0,18180.0,18132.0,NaN,NaN
9996,18180.0,40.761806,-73.829559,40.692406,-73.745216,40.692326,-73.744289,318.0,18240.0,15733.0,NaN,NaN
9997,18180.0,40.761806,-73.829559,40.692406,-73.745216,40.692326,-73.744289,318.0,18240.0,15625.0,NaN,NaN
9998,18180.0,40.862228,-73.890442,40.877934,-73.866585,40.871502,-73.876412,185.0,18180.0,18085.0,NaN,NaN


In [22]:
df.dropna(axis = 0, how = 'any', thresh = None, subset = None, inplace = True) #The code, if we want to remove all the rows with empty cells.  
df

,RecordedAtTime,OriginLat,OriginLong,DestinationLat,DestinationLong,VehicleLocation.Latitude,VehicleLocation.Longitude,DistanceFromStop,ExpectedArrivalTime,ScheduledArrivalTime,O_V,V_D
0,180.0,40.616104,-74.031143,40.656048,-73.907379,40.635170,-73.960803,76.0,180.0,374.0,6303.540673,6303.540673
1,180.0,40.643169,-74.073494,40.575935,-74.167686,40.590802,-74.158340,62.0,180.0,86282.0,9230.043179,9230.043179
2,180.0,40.875008,-73.880142,40.912376,-73.902534,40.886010,-73.912647,5.0,180.0,53.0,2994.093405,2994.093405
3,180.0,40.701748,-73.802399,40.666012,-73.735939,40.668002,-73.729348,267.0,240.0,180.0,7212.611156,7212.611156
4,180.0,40.881187,-73.909340,40.809654,-73.928360,40.868134,-73.893032,11.0,180.0,86378.0,1996.682927,1996.682927
...,...,...,...,...,...,...,...,...,...,...,...,...
8928,16440.0,40.861012,-73.890450,40.816143,-73.917549,40.860429,-73.890861,160.0,16440.0,16839.0,4469.447811,4469.447811
8929,16380.0,40.642979,-73.878334,40.576973,-73.981400,40.620067,-73.941519,158.0,16440.0,16558.0,10550.759498,10550.759498
8930,16380.0,40.642979,-73.878334,40.576973,-73.981400,40.620067,-73.941519,158.0,16440.0,16556.0,10550.759498,10550.759498
8931,16380.0,40.586937,-74.086418,40.643570,-74.073227,40.621926,-74.078227,43.0,16440.0,16282.0,1566.934347,1566.934347


In [23]:
df['delay']=abs(df['ExpectedArrivalTime'] - df['ScheduledArrivalTime'])
df


,RecordedAtTime,OriginLat,OriginLong,DestinationLat,DestinationLong,VehicleLocation.Latitude,VehicleLocation.Longitude,DistanceFromStop,ExpectedArrivalTime,ScheduledArrivalTime,O_V,V_D,delay
0,180.0,40.616104,-74.031143,40.656048,-73.907379,40.635170,-73.960803,76.0,180.0,374.0,6303.540673,6303.540673,194.0
1,180.0,40.643169,-74.073494,40.575935,-74.167686,40.590802,-74.158340,62.0,180.0,86282.0,9230.043179,9230.043179,86102.0
2,180.0,40.875008,-73.880142,40.912376,-73.902534,40.886010,-73.912647,5.0,180.0,53.0,2994.093405,2994.093405,127.0
3,180.0,40.701748,-73.802399,40.666012,-73.735939,40.668002,-73.729348,267.0,240.0,180.0,7212.611156,7212.611156,60.0
4,180.0,40.881187,-73.909340,40.809654,-73.928360,40.868134,-73.893032,11.0,180.0,86378.0,1996.682927,1996.682927,86198.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8928,16440.0,40.861012,-73.890450,40.816143,-73.917549,40.860429,-73.890861,160.0,16440.0,16839.0,4469.447811,4469.447811,399.0
8929,16380.0,40.642979,-73.878334,40.576973,-73.981400,40.620067,-73.941519,158.0,16440.0,16558.0,10550.759498,10550.759498,118.0
8930,16380.0,40.642979,-73.878334,40.576973,-73.981400,40.620067,-73.941519,158.0,16440.0,16556.0,10550.759498,10550.759498,116.0
8931,16380.0,40.586937,-74.086418,40.643570,-74.073227,40.621926,-74.078227,43.0,16440.0,16282.0,1566.934347,1566.934347,158.0


In [24]:
df['duration']=abs(df['ExpectedArrivalTime'] - df['RecordedAtTime'])
df

,RecordedAtTime,OriginLat,OriginLong,DestinationLat,DestinationLong,VehicleLocation.Latitude,VehicleLocation.Longitude,DistanceFromStop,ExpectedArrivalTime,ScheduledArrivalTime,O_V,V_D,delay,duration
0,180.0,40.616104,-74.031143,40.656048,-73.907379,40.635170,-73.960803,76.0,180.0,374.0,6303.540673,6303.540673,194.0,0.0
1,180.0,40.643169,-74.073494,40.575935,-74.167686,40.590802,-74.158340,62.0,180.0,86282.0,9230.043179,9230.043179,86102.0,0.0
2,180.0,40.875008,-73.880142,40.912376,-73.902534,40.886010,-73.912647,5.0,180.0,53.0,2994.093405,2994.093405,127.0,0.0
3,180.0,40.701748,-73.802399,40.666012,-73.735939,40.668002,-73.729348,267.0,240.0,180.0,7212.611156,7212.611156,60.0,60.0
4,180.0,40.881187,-73.909340,40.809654,-73.928360,40.868134,-73.893032,11.0,180.0,86378.0,1996.682927,1996.682927,86198.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8928,16440.0,40.861012,-73.890450,40.816143,-73.917549,40.860429,-73.890861,160.0,16440.0,16839.0,4469.447811,4469.447811,399.0,0.0
8929,16380.0,40.642979,-73.878334,40.576973,-73.981400,40.620067,-73.941519,158.0,16440.0,16558.0,10550.759498,10550.759498,118.0,60.0
8930,16380.0,40.642979,-73.878334,40.576973,-73.981400,40.620067,-73.941519,158.0,16440.0,16556.0,10550.759498,10550.759498,116.0,60.0
8931,16380.0,40.586937,-74.086418,40.643570,-74.073227,40.621926,-74.078227,43.0,16440.0,16282.0,1566.934347,1566.934347,158.0,60.0


In [25]:
df["DistanceFromStop"] = pd.to_numeric(df["DistanceFromStop"], errors='coerce').fillna(0, downcast='infer')

In [26]:
print(df.dtypes)

RecordedAtTime               float64
OriginLat                    float64
OriginLong                   float64
DestinationLat               float64
DestinationLong              float64
VehicleLocation.Latitude     float64
VehicleLocation.Longitude    float64
DistanceFromStop               int64
ExpectedArrivalTime          float64
ScheduledArrivalTime         float64
O_V                          float64
V_D                          float64
delay                        float64
duration                     float64
dtype: object


In [27]:
variables = ['RecordedAtTime','DistanceFromStop', 'O_V', 'V_D', 'ExpectedArrivalTime', 'ScheduledArrivalTime', 'delay', 'duration']
df_new = df[variables]
df_new.head()

,RecordedAtTime,DistanceFromStop,O_V,V_D,ExpectedArrivalTime,ScheduledArrivalTime,delay,duration
0,180.0,76,6303.540673,6303.540673,180.0,374.0,194.0,0.0
1,180.0,62,9230.043179,9230.043179,180.0,86282.0,86102.0,0.0
2,180.0,5,2994.093405,2994.093405,180.0,53.0,127.0,0.0
3,180.0,267,7212.611156,7212.611156,240.0,180.0,60.0,60.0
4,180.0,11,1996.682927,1996.682927,180.0,86378.0,86198.0,0.0


In [28]:
df_new

,RecordedAtTime,DistanceFromStop,O_V,V_D,ExpectedArrivalTime,ScheduledArrivalTime,delay,duration
0,180.0,76,6303.540673,6303.540673,180.0,374.0,194.0,0.0
1,180.0,62,9230.043179,9230.043179,180.0,86282.0,86102.0,0.0
2,180.0,5,2994.093405,2994.093405,180.0,53.0,127.0,0.0
3,180.0,267,7212.611156,7212.611156,240.0,180.0,60.0,60.0
4,180.0,11,1996.682927,1996.682927,180.0,86378.0,86198.0,0.0
...,...,...,...,...,...,...,...,...
8928,16440.0,160,4469.447811,4469.447811,16440.0,16839.0,399.0,0.0
8929,16380.0,158,10550.759498,10550.759498,16440.0,16558.0,118.0,60.0
8930,16380.0,158,10550.759498,10550.759498,16440.0,16556.0,116.0,60.0
8931,16380.0,43,1566.934347,1566.934347,16440.0,16282.0,158.0,60.0


In [29]:
X = df_new.iloc[:, 1:4].values
Y = df_new.iloc[:, 7].values
Y

array([ 0.,  0.,  0., ..., 60., 60., 60.])

In [30]:
X

array([[7.60000000e+01, 6.30354067e+03, 6.30354067e+03],
       [6.20000000e+01, 9.23004318e+03, 9.23004318e+03],
       [5.00000000e+00, 2.99409341e+03, 2.99409341e+03],
       ...,
       [1.58000000e+02, 1.05507595e+04, 1.05507595e+04],
       [4.30000000e+01, 1.56693435e+03, 1.56693435e+03],
       [1.45000000e+02, 7.76298703e+03, 7.76298703e+03]])

In [31]:
Y.shape

(7959,)

In [32]:
X.shape

(7959, 3)

In [33]:
#X.reshape(-1,1)

In [34]:
X.shape

(7959, 3)

In [35]:
scaler = MinMaxScaler()
X= scaler.fit_transform(X)
X

array([[0.00831783, 0.20282799, 0.20282799],
       [0.0067856 , 0.29707273, 0.29707273],
       [0.00054723, 0.09625094, 0.09625094],
       ...,
       [0.01729233, 0.33960492, 0.33960492],
       [0.00470614, 0.05029087, 0.05029087],
       [0.01586954, 0.24982783, 0.24982783]])

In [36]:
X = torch.tensor(X)
Y = torch.tensor(Y)
X.shape

torch.Size([7959, 3])

In [37]:
X

tensor([[0.0083, 0.2028, 0.2028],
        [0.0068, 0.2971, 0.2971],
        ...,
        [0.0047, 0.0503, 0.0503],
        [0.0159, 0.2498, 0.2498]], dtype=torch.float64)

In [38]:
#X =X.unsqueeze(1)

In [39]:
n_samples = X.shape[0]
n_val = int(0.2 * n_samples)

shuffled_indices = torch.randperm(n_samples)

train_indices = shuffled_indices[:-n_val]
val_indices = shuffled_indices[-n_val:]

train_X = X[train_indices]
train_Y = Y[train_indices]
#train_Y = train_Y.unsqueeze(1)

val_X = X[val_indices]
val_Y = Y[val_indices]
#val_Y = val_Y.unsqueeze(1)

In [40]:
print(train_X.shape)
print(train_Y.shape)
print(val_X.shape)
print(val_Y.shape)

torch.Size([6368, 3])
torch.Size([6368])
torch.Size([1591, 3])
torch.Size([1591])


In [41]:
train_X = train_X.float()
train_Y = train_Y.float()
val_X = val_X.float()
val_Y = val_Y.float()
print(train_X.dtype)
print(train_Y.dtype)
print(val_X.dtype)
print(val_Y.dtype)

torch.float32
torch.float32
torch.float32
torch.float32


In [42]:
from collections import OrderedDict

seq_model = nn.Sequential(OrderedDict([
    ('hidden_linear', nn.Linear(3, 10)),
    ('hidden_activation', nn.ReLU()), 
     ('hidden_linear2', nn.Linear(10, 6)),
    ('hidden_activation2', nn.ReLU()),
    ('output_linear', nn.Linear(6, 1))
]))

seq_model

Sequential(
  (hidden_linear): Linear(in_features=3, out_features=10, bias=True)
  (hidden_activation): ReLU()
  (hidden_linear2): Linear(in_features=10, out_features=6, bias=True)
  (hidden_activation2): ReLU()
  (output_linear): Linear(in_features=6, out_features=1, bias=True)
)

In [43]:
def training_loop(n_epochs, optimizer, model, loss_fn, train_X, val_X, train_Y, val_Y):
    
    for epoch in range(1, n_epochs + 1):
        train_t_p = model(train_X) 
        train_loss = loss_fn(train_t_p, train_Y)
                             
        val_t_p = model(val_X) 
        val_loss = loss_fn(val_t_p, val_Y)
        
        optimizer.zero_grad()
        train_loss.backward() 
        optimizer.step()

        if epoch ==1 or epoch % 50==0:
            print(f"Epoch {epoch}, Training loss {train_loss.item():.4f},"
                  f" Validation loss {val_loss.item():.4f}")

In [44]:
optimizer= optim.Adam(seq_model.parameters(), lr=1e-2)

training_loop(
    n_epochs=500,
    optimizer= optimizer,
    model=seq_model,
    loss_fn=nn.MSELoss(),
    train_X=train_X,
    val_X= val_X,
    train_Y= train_Y,
    val_Y= val_Y)

C:\Users\Sanaz\anaconda3\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([6368])) that is different to the input size (torch.Size([6368, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
C:\Users\Sanaz\anaconda3\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([1591])) that is different to the input size (torch.Size([1591, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 1, Training loss 4476.9102, Validation loss 3883.8691
Epoch 50, Training loss 3805.6228, Validation loss 3265.6101
Epoch 100, Training loss 2853.1384, Validation loss 2483.4465
Epoch 150, Training loss 2823.9690, Validation loss 2444.7258
Epoch 200, Training loss 2811.3315, Validation loss 2434.7859
Epoch 250, Training loss 2804.1516, Validation loss 2429.6770
Epoch 300, Training loss 2800.7778, Validation loss 2427.5281
Epoch 350, Training loss 2799.4902, Validation loss 2426.8999
Epoch 400, Training loss 2799.0833, Validation loss 2426.8267
Epoch 450, Training loss 2798.9702, Validation loss 2426.8748
Epoch 500, Training loss 2798.9390, Validation loss 2426.9202
